In [59]:
from langchain_core.documents import Document

In [60]:
document = [
    Document(
        page_content = "Dogs are great companions, known for their loyalty and friendliness.",
        meta_data={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content = "Cats are independent pets that ofteb enjoy their own space.",
        meta_data={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content = "Goldfish are popular pets for beginners, requiring relatively simple care.",
        meta_data={"source":"fish-pets-doc"},
    ),
    Document(
        page_content = "Parrots are intelligent birds capable of mimicking human speech.",
        meta_data={"source":"bird-pets-doc"},
    ),
    Document(
        page_content = "Rabbits are social animals that need plenty of space to hop around.",
        meta_data={"source":"mammal-pets-doc"},
    ),
]

In [61]:
## vector stores
from langchain_chroma import Chroma

In [62]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [63]:
from langchain_groq import ChatGroq

In [64]:
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")

In [65]:
from langchain_huggingface import HuggingFaceEmbeddings

In [66]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [67]:
vectorstore = Chroma.from_documents(document,embedding=embeddings)

In [68]:
vectorstore.similarity_search("cat")

[Document(id='931411aa-ee10-4ff3-aaec-34b57e970ff3', metadata={}, page_content='Cats are independent pets that ofteb enjoy their own space.'),
 Document(id='27b22983-ea31-4ba3-a63c-95f168a4974f', metadata={}, page_content='Cats are independent pets that ofteb enjoy their own space.'),
 Document(id='3f0071bf-87d9-4996-8da8-9f11e770a8bf', metadata={}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='83002884-6c96-4502-a44a-feaf5f67242f', metadata={}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [69]:
##Async query
await vectorstore.asimilarity_search("cat")

[Document(id='931411aa-ee10-4ff3-aaec-34b57e970ff3', metadata={}, page_content='Cats are independent pets that ofteb enjoy their own space.'),
 Document(id='27b22983-ea31-4ba3-a63c-95f168a4974f', metadata={}, page_content='Cats are independent pets that ofteb enjoy their own space.'),
 Document(id='3f0071bf-87d9-4996-8da8-9f11e770a8bf', metadata={}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='83002884-6c96-4502-a44a-feaf5f67242f', metadata={}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [70]:
vectorstore.similarity_search_with_score("Cat")

[(Document(id='931411aa-ee10-4ff3-aaec-34b57e970ff3', metadata={}, page_content='Cats are independent pets that ofteb enjoy their own space.'),
  0.9372676610946655),
 (Document(id='27b22983-ea31-4ba3-a63c-95f168a4974f', metadata={}, page_content='Cats are independent pets that ofteb enjoy their own space.'),
  0.9372676610946655),
 (Document(id='3f0071bf-87d9-4996-8da8-9f11e770a8bf', metadata={}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(id='83002884-6c96-4502-a44a-feaf5f67242f', metadata={}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956)]

In [71]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

In [72]:
## creating retriver with Runnable Lambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='27b22983-ea31-4ba3-a63c-95f168a4974f', metadata={}, page_content='Cats are independent pets that ofteb enjoy their own space.')],
 [Document(id='83002884-6c96-4502-a44a-feaf5f67242f', metadata={}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [73]:
## creating retriever with vectorstore retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])

[[Document(id='27b22983-ea31-4ba3-a63c-95f168a4974f', metadata={}, page_content='Cats are independent pets that ofteb enjoy their own space.')],
 [Document(id='83002884-6c96-4502-a44a-feaf5f67242f', metadata={}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [74]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [75]:
message = """
Answer this question using the provided context only.

{question}

context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human",message)])

In [76]:
from langchain_core.output_parsers import StrOutputParser

In [77]:
parser = StrOutputParser()

In [78]:
rag_chain = {"context":retriever,"question":RunnablePassthrough()}|prompt|llm|parser

In [79]:
response = rag_chain.invoke("tell me about dogs")

In [80]:
response

'According to the provided context, dogs are great companions, known for their loyalty and friendliness.'

In [81]:
## way 2 -- one more way of creating advanced rag chain

In [82]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [83]:
retriever# for vector store

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000022F63C0C9D0>, search_kwargs={'k': 1})

In [84]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "use the following pieces of retrieved context to answer "
    "the question. if you don't know the answer, say that you "
    "don't know, use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages([("system",system_prompt),
                                          ("human","{input}"),
                                          ])
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [85]:
response = rag_chain.invoke({'input':"tell me about dog"})

In [86]:
response

{'input': 'tell me about dog',
 'context': [Document(id='83002884-6c96-4502-a44a-feaf5f67242f', metadata={}, page_content='Dogs are great companions, known for their loyalty and friendliness.')],
 'answer': 'According to the given context, dogs are known for their loyalty and friendliness, making them great companions.'}

In [87]:
response['answer']

'According to the given context, dogs are known for their loyalty and friendliness, making them great companions.'

In [88]:
#adding chat history

In [89]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [90]:
contextualizr_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do not answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

In [91]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualizr_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

In [92]:
history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

In [94]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualizr_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

In [95]:
question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

ValueError: Prompt must accept context as an input variable. Received prompt with input variables: ['chat_history', 'input']